In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from plots import histogram
from loss import kl_divergence
from stock_price_handler import load_stock_prices, get_log_returns
from sde import DeepSDE

In [2]:
TIME_INCREMENT = 5
STOCK_NAME = "apple"

In [3]:
stock_prices = load_stock_prices(STOCK_NAME)
actual_log_returns, stock_prices = get_log_returns(stock_prices)

In [4]:
sigma = np.std(actual_log_returns) / np.sqrt(TIME_INCREMENT)
mu = np.mean(actual_log_returns) / TIME_INCREMENT + np.square(sigma) / 2
initial_states = torch.tensor(np.array(stock_prices)).to(torch.float32).unsqueeze(-1)

In [5]:
black_scholes = DeepSDE(hidden_size=1, mu=mu, sigma=sigma, sample_size=len(actual_log_returns), times=torch.linspace(0, TIME_INCREMENT, TIME_INCREMENT), desired_log_returns=torch.tensor(actual_log_returns))
optimizer = torch.optim.Adam(black_scholes.parameters(), lr=1e-4)

In [6]:
def train_loop(model, loss_function, optimizer, num_epochs):

    model.train()
    for epoch in range(num_epochs):

        estimated_log_returns = model(initial_states)
        loss = loss_function(estimated_log_returns, torch.tensor(actual_log_returns))

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        histogram(actual_log_returns, label="Actual", color="black")
        histogram(estimated_log_returns.detach().numpy(), label="Black Scholes", color="orange")
        plt.title(f"Epoch {epoch}: Loss = {loss.item():>.7f}")
        plt.legend()
        plt.show()


In [7]:
train_loop(black_scholes, kl_divergence, optimizer, 50)

ValueError: `y0` must be a 2-dimensional tensor of shape (batch, channels).